In [5]:
# pipeline_retrieval_bankqa.py
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1️⃣ Đọc file dataset
with open("/content/drive/MyDrive/bank_qa_laisuat_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 2️⃣ Trích xuất toàn bộ văn bản từ các phần khác nhau
def extract_text(data):
    documents = []

    for item in data:
        # --- Bài báo ---
        if isinstance(item, dict) and "content" in item:
            documents.append(item["content"])

            # Thêm QA pairs nếu có
            if "qa_pairs" in item:
                for qa in item["qa_pairs"]:
                    documents.append(f"{qa['question']} {qa['answer']}")

        # --- Công điện ---
        elif isinstance(item, dict) and "document" in item:
            doc = item["document"]
            ctx = item.get("context", {})
            directives = item.get("directives", {})
            merged_text = f"{doc['title']} {doc.get('purpose', '')} {ctx.get('background', '')}"
            documents.append(merged_text)

        # --- Báo cáo NHNN ---
        elif isinstance(item, dict) and "summary" in item:
            documents.append(item["summary"])

        # --- Mảng QA độc lập ---
        elif isinstance(item, list):
            for qa in item:
                if "question" in qa and "answer" in qa:
                    context = qa.get("context", "") # Get context, default to empty string if not present
                    documents.append(f"{context} {qa['question']} {qa['answer']}")

    return documents

documents = extract_text(data)

# 3️⃣ Xây TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words=None)
tfidf_matrix = vectorizer.fit_transform(documents)

# 4️⃣ Hàm truy vấn
def retrieve(query, top_k=3):
    query_vec = vectorizer.transform([query])
    sims = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = sims.argsort()[-top_k:][::-1]
    results = [(documents[i], float(sims[i])) for i in top_indices]
    return results

# 5️⃣ Thử nghiệm
if __name__ == "__main__":
    query = input("Nhập câu hỏi: ")
    results = retrieve(query)
    print("\n🔍 Kết quả truy xuất:")
    for i, (text, score) in enumerate(results, 1):
        print(f"\n--- Kết quả {i} (độ tương đồng = {score:.4f}) ---")
        print(text[:500], "...")

Nhập câu hỏi: Những ngân hàng nào đã tăng lãi suất đầu năm 2025?

🔍 Kết quả truy xuất:

--- Kết quả 1 (độ tương đồng = 0.5296) ---
Những ngân hàng nào đã tăng lãi suất huy động đầu năm 2025? Ít nhất 7 ngân hàng đã tăng lãi suất huy động gồm Agribank, Bac A Bank, NCB, Eximbank, KienlongBank... chủ yếu ở kỳ hạn dài từ 12 tháng trở lên. ...

--- Kết quả 2 (độ tương đồng = 0.4759) ---
 Có bao nhiêu ngân hàng đã tăng lãi suất huy động từ đầu tháng 11? 13 ngân hàng ...

--- Kết quả 3 (độ tương đồng = 0.3296) ---
Ngay từ đầu năm 2025, nhiều NHTM đã tăng lãi suất huy động tiền gửi, triển khai các chương trình ưu đãi để thu hút khách hàng gửi tiết kiệm. Từ đầu tháng 1/2025 đến nay, đã có ít nhất 7 ngân hàng tăng lãi suất huy động trong đó có Agribank, Bac A Bank, NCB, Eximbank, KienlongBank... Các ngân hàng này chủ yếu điều chỉnh tăng lãi suất ở các kỳ hạn dài từ 12 tháng trở lên. Eximbank hiện đang áp dụng mức lãi suất từ 6,5% - 6,8%/năm cho các khoản tiền gửi có kỳ hạn từ 15 đến 34 tháng. Đây